In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
#export
import os
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from am4894bq.schema import get_schema, df_to_bq_schema, schema_diff, update_bq_schema, update_df_schema
from am4894bq.utils import does_table_exist

load_dotenv()

bq_project_id = os.getenv('BQ_PROJECT_ID')

In [ ]:
# default_exp pd

# pd

> Pandas related BigQuery functionality.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export


def df_to_gbq(df: pd.DataFrame, destination_table: str, project_id: str, if_exists: str = 'append', print_info: bool = True, mode: str = 'pandas') -> pd.DataFrame:
    """
    Save df to BigQuery enforcing schema consistency between df and destination table if it exists.
    """
    
    # only do anything if mode set to wrangle, otherwise just use pandas
    if mode == 'wrangle':
    
        table_id = f'{project_id}.{destination_table}'
        bq_client = bigquery.Client()

        # only need to handle schema's if table already exists and if_exists != 'replace'
        if does_table_exist(bq_client, table_id) and if_exists != 'replace' :

            old_schema = get_schema(table_id)
            new_schema = df_to_bq_schema(df)
            diffs = schema_diff(old_schema, new_schema)

            if len(diffs) > 0:

                # update the table schema in BigQuery
                update_bq_schema(bq_client, table_id, diffs, print_info=print_info)

                # update the df schema to be as expected by BigQuery
                df = update_df_schema(bq_client, table_id, diffs, df, print_info=print_info)
        
    # load to BigQuery
    df.to_gbq(destination_table, project_id=project_id, if_exists=if_exists)
            
    return df



In [ ]:
#hide

# tests

# make a dummy df
df = pd.DataFrame([['a0', 'b0'],['a1', 'b1']], columns=['col_a', 'col_b'])

# send to bq
df = df_to_gbq(df, 'tmp.tmp', project_id=bq_project_id, if_exists='replace', mode='wrangle')

# read back from bq
df_bq = pd.read_gbq("select * from tmp.tmp")

assert str(df) == str(df_bq)

# add a new col to df
df['col_c'] = ['c0', 'c1']

# drop col_b
df = df.drop(['col_b'], axis=1)

# save to bq
df = df_to_gbq(df, 'tmp.tmp', project_id=bq_project_id, if_exists='append', print_info=False, mode='wrangle')

# read back from bq
df_bq = pd.read_gbq("select * from tmp.tmp")

assert str(df_bq) == '  col_a col_b col_c\n0    a0    b0  None\n1    a1    b1  None\n2    a0  None    c0\n3    a1  None    c1'

1it [00:04,  4.51s/it]
Downloading: 100%|██████████| 2/2 [00:00<00:00,  9.56rows/s]
1it [00:02,  2.97s/it]
Downloading: 100%|██████████| 4/4 [00:00<00:00, 21.72rows/s]
